In [1]:
import pandas as pd
import numpy as np
import time
import os

t = time.localtime()
timestamp = time.strftime('%y%m%d-%H%M', t)

# read excel
df = pd.read_excel('H://Current_Working_Folder//Reports Analytics//202001//202001-t_worc_families-20200511_1617.xlsx')

# assign path and reporitng month
drive = 'H://Current_Working_Folder//Reports Analytics//'
reporting_month = str(df.loc[0,'reporting_month'])

# Filter with column names ending with amt, amount, nbr_month
df_amt = df.filter(regex = 'amt$|amount$|amount$|nbr_month$|nbr_months', axis='columns')

# Describe the filtered dataframe
df_des = df_amt.describe()

# Write the describe to excel
writer = pd.ExcelWriter(drive + reporting_month + '//' + reporting_month + '_' + 'report' + '-' + timestamp + '.xlsx')
#df_des.to_excel(writer, index = False)
df_des.to_excel(writer, sheet_name='family_stat')
writer.save()

In [2]:
df.columns

Index(['id', 'category', 'reporting_month', 'case_number', 'county_fips_code',
       'stratum', 'zip_code', 'funding_stream', 'disposition', 'new_applicant',
       'family_size', 'family_type', 'subsid_housing', 'medical_assist',
       'food_stamps', 'food_stamps_amt', 'subsid_child_care',
       'subsid_child_care_amt', 'child_support_amt', 'cash_resource_amt',
       'cash & cash equivalents_amount', 'cash & cash equivalents_nbr_month',
       'tanf child care_amount', 'tanf child care_children_covered',
       'tanf child care_nbr_months', 'transportation_amount',
       'transportation_nbr_months', 'transitional services_amount',
       'transitional services_nbr_months', 'other_amount', 'other_nbr_months',
       'reason for amount of reduction_sanction reduction_amount',
       'reason for amount of reduction_work requirements sanction',
       'reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent',
       'reason for amount of 

In [3]:
df.shape

(2280, 56)

In [4]:
df['subsid_child_care'].value_counts()

No subsidized child care received                                                                                                                 2181
Yes, receives child care funded entirely or in part with Federal funds (e.g., receives TANF, CCDF, SSBG, or other federally funded child care)      99
Name: subsid_child_care, dtype: int64

In [5]:
df['subsid_child_care_amt'].sum()

81730

In [6]:
df['tanf child care_children_covered'].value_counts()

0    2280
Name: tanf child care_children_covered, dtype: int64

In [7]:
df['tanf child care_amount'].sum()

0

In [8]:
cat = df.drop(df_amt, axis='columns')
#cat1 = cat.drop((cat.loc[:,'reporting_month':'stratum']),axis='columns')
cat1 = cat.drop(['id','reporting_month','case_number','county_fips_code','stratum','disposition','worker_id',
                 'case_manager','area','created_by','updated_by','created_at','updated_at','ans_county',
                 'local_office','waiver_grps'], axis='columns')
cat2 = cat1.drop(cat1.filter(regex = '^reason|^other'), axis='columns')

In [9]:
reason_cat = cat1.filter(regex = '^reason', axis='columns')

for c in reason_cat.columns:
    print(reason_cat[c].value_counts())
    print('---')


No     2044
Yes     236
Name: reason for amount of reduction_work requirements sanction, dtype: int64
---
No    2280
Name: reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent, dtype: int64
---
No    2280
Name: reason for amount of reduction_Sanction for Teen Parent not Attending School, dtype: int64
---
No     2165
Yes     115
Name: reason for amount of reduction_Non-Cooperation with Child Support, dtype: int64
---
No     2279
Yes       1
Name: reason for amount of reduction_Failure to Comply with an Individual Responsibility Plan, dtype: int64
---
No     2275
Yes       5
Name: reason for amount of reduction_Other Sanction, dtype: int64
---
0    2280
Name: reason for amount of reduction_Recoupment of Prior Overpayment, dtype: int64
---


In [10]:
#((df.filter(regex = '^reason'))).head(20)

In [11]:
for c in cat2.columns:
    print((cat2[c].value_counts().to_frame('count').rename_axis(c)).reset_index())
    print('---')

   category  count
0       Tea   2045
1  WorkPays    235
---
     zip_code  count
0       71854     81
1       72301     70
2       72209     65
3       72401     55
4       72390     39
5       72335     39
6       72114     38
7       72143     38
8       72956     37
9       72204     37
10      72764     36
11      72315     35
12      72076     33
13      72360     32
14      71603     32
15      71601     31
16      72206     30
17      72342     30
18      72904     30
19      72901     29
20      72117     28
21      72450     27
22      71701     26
23      71730     25
24      72032     23
25      72703     22
26      72701     22
27      71913     22
28      71953     21
29      72601     21
..        ...    ...
334     72579      1
335     72567      1
336     72565      1
337     72523      1
338     72863      1
339     74960      1
340     72935      1
341     72941      1
342     72052      1
343     72036      1
344     72030      1
345     72020      1
346     72010  

In [12]:
worc_fam = df
fam_type = worc_fam.groupby(['category','family_type']).agg({'case_number' : 'nunique'}).rename({'case_number': 'case_count'}, axis='columns')#.reset_index()
fam_type

case_count
category family_type                                                   
Tea      Family included only in overall work participat...         907
         Family with no work-eligible individual                   1084
         Two-Parent Family included in both the overall ...          54
WorkPays Family included only in overall work participat...         224
         Two-Parent Family included in both the overall ...          11